In [1]:
# Let's perform KG-RAG on a mock Obsidian vault
#   - By leveraging Graphiti, we're hoping to see improvements over traditional RAG!

In [2]:
import asyncio

from dotenv import load_dotenv

load_dotenv()  # load environment variables

True

In [3]:
# Init: Set up logging and environment variables for connecting to the Neo4j database

import logging
import os
from logging import INFO

# Configure logging
logging.basicConfig(
    level=INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
)
logger = logging.getLogger(__name__)

# Neo4j connection parameters
neo4j_uri = os.environ.get('NEO4J_URI', 'bolt://localhost:7687')
neo4j_user = os.environ.get('NEO4J_USER', 'neo4j')
neo4j_password = os.environ.get('NEO4J_PASSWORD', "password")

if not neo4j_uri or not neo4j_user or not neo4j_password:
    raise ValueError('NEO4J_URI, NEO4J_USER, and NEO4J_PASSWORD must be set')

In [4]:
# Init: Configure Graphiti to work with Ollama

from graphiti_core import Graphiti
from graphiti_core.llm_client.config import LLMConfig
from graphiti_core.llm_client.openai_client import OpenAIClient
from graphiti_core.embedder.openai import OpenAIEmbedder, OpenAIEmbedderConfig
from graphiti_core.cross_encoder.openai_reranker_client import OpenAIRerankerClient

llm_config = LLMConfig(
    api_key="abc",  # Ollama doesn't require a real API key
    model="gemma3n",
    small_model="gemma3n",
    base_url="http://localhost:11434/v1",  # Ollama provides this port
)
llm_client = OpenAIClient(config=llm_config)

graphiti = Graphiti(
    neo4j_uri,
    neo4j_user,
    neo4j_password,
    llm_client=llm_client,
    embedder=OpenAIEmbedder(
        config=OpenAIEmbedderConfig(
            api_key="abc",
            embedding_model="nomic-embed-text",
            embedding_dim=768,
            base_url="http://localhost:11434/v1",
        )
    ),
    cross_encoder=OpenAIRerankerClient(client=llm_client, config=llm_config),
)

In [5]:
# Init: Clear Neo4J and initialize Graphiti

# Delete all nodes and edges in Neo4j
await graphiti.driver.execute_query("MATCH (n) DETACH DELETE n")

# Initialize the graph database with graphiti's indices. This only needs to be done once.
await graphiti.build_indices_and_constraints()

2025-08-06 18:51:40 - neo4j.notifications - INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE RANGE INDEX entity_uuid IF NOT EXISTS FOR (e:Entity) ON (e.uuid)` has no effect.} {description: `RANGE INDEX entity_uuid FOR (e:Entity) ON (e.uuid)` already exists.} {position: None} for query: 'CREATE INDEX entity_uuid IF NOT EXISTS FOR (n:Entity) ON (n.uuid)'
2025-08-06 18:51:40 - neo4j.notifications - INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE RANGE INDEX episode_uuid IF NOT EXISTS FOR (e:Episodic) ON (e.uuid)` has no effect.} {description: `RANGE INDEX episode_uuid FOR (e:Episodic) ON (e.uuid)` already exists.} {position: None} for query: 'CREATE INDEX episode_uuid IF NOT EXISTS FOR (n:Episodic) ON (n.uuid)'
2025-08-06 18:51:40 - neo4j.n

In [6]:
# Indexing: Let's load our full documents

from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader("./", glob="**/*.md", show_progress=True, use_multithreading=True)

full_documents = loader.load()
full_documents[0]

  0%|          | 0/66 [00:00<?, ?it/s]2025-08-06 18:51:43 - unstructured - WARNING - Need to load profiles.
2025-08-06 18:51:43 - unstructured - WARNING - Need to load profiles.
2025-08-06 18:51:43 - unstructured - WARNING - Need to load profiles.
2025-08-06 18:51:43 - unstructured - WARNING - Need to load profiles.
2025-08-06 18:51:43 - unstructured - WARNING - short text: "![[priority_queue_overview.png|500]]". Defaulting to English.
2025-08-06 18:51:43 - unstructured - WARNING - short text: "![[integral_image_overview.png]]". Defaulting to English.
2025-08-06 18:51:43 - unstructured - WARNING - short text: "Operations". Defaulting to English.
2025-08-06 18:51:43 - unstructured - WARNING - short text: "Operation". Defaulting to English.
2025-08-06 18:51:43 - unstructured - WARNING - short text: "Multiply by two:". Defaulting to English.
2025-08-06 18:51:43 - unstructured - WARNING - short text: "x <<= 1". Defaulting to English.
2025-08-06 18:51:43 - unstructured - WARNING - short tex

Document(metadata={'source': 'rsc/vault/Priority Queue.md'}, page_content='A priority queue is an abstract data structure, similar to a queue, in which each element has an associated priority and elements with high priority are served before elements with low priority.\n\n![[priority_queue_overview.png|500]]\n\nPriority queues are commonly implemented using [[Heap|heaps]], giving $O(\\log n)$ performance for inserts and removals, and $O(n)$ to build the heap initially from a set of $n$ elements.\n\nOperations\n\nPriority queues support the following operations:\n\nBasic - enqueue: add an element to the queue with an associated priority. - dequeue: remove the highest priority element from the queue, and return it. - delete: remove an element from the queue. - peek: return the highest priority element from the queue.\n\nInspection - size: return the number of elements in the queue. - is_empty: check whether the queue has no elements.\n\nEquivalence of priority queues and sorting algorith

In [7]:
# Indexing: Ingest chunked documents as episodes into Graphiti
#   - Sooooooooooo slooooooooooow... and it breaks with Ollama models, because they're too weak
#   - So you've got to have a ChatGPT subscription and pay a million dollars to build a knowledge graph

from graphiti_core.nodes import EpisodeType
from langchain_text_splitters import RecursiveCharacterTextSplitter
from datetime import datetime, timezone

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
documents = text_splitter.split_documents(full_documents)

for i in range(5):
    doc = documents[i]
    print(f'Adding episode... Obsidian chunk {i} ({i + 1}/{len(documents)})')
    await graphiti.add_episode(
        name=f'Obsidian chunk {i}',
        episode_body=doc.page_content,
        source=EpisodeType.text,
        source_description=f"Obsidian: {doc.metadata['source']}",
        reference_time=datetime.now(timezone.utc),
    )

Adding episode... Obsidian chunk 0 (1/305)


2025-08-06 18:51:46 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 18:51:46 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-06 18:51:51 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 18:51:58 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 18:51:58 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-06 18:51:58 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-06 18:52:01 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 18:52:01 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-06 18:52:01 - graphiti_core.graphiti - INFO - Completed add_episode in 17237.697

Adding episode... Obsidian chunk 1 (2/305)


2025-08-06 18:52:06 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 18:52:06 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-06 18:52:06 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-06 18:52:06 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-06 18:52:06 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-06 18:52:06 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-06 18:52:22 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 18:52:41 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 18:52:41 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "

Adding episode... Obsidian chunk 2 (3/305)


...

2025-08-06 19:28:43 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 19:28:43 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-06 19:28:43 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-06 19:28:43 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-06 19:28:43 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-06 19:28:43 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-06 19:28:43 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-06 19:28:43 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-06 19:28:43 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200

Adding episode... Obsidian chunk 8 (9/305)


2025-08-06 19:32:39 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 19:32:39 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-06 19:32:39 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-06 19:32:39 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-06 19:32:39 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-06 19:33:10 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 19:33:21 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 19:33:21 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-06 19:33:27 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/complet

Adding episode... Obsidian chunk 9 (10/305)


2025-08-06 19:34:25 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 19:34:25 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-06 19:34:25 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-06 19:34:25 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-06 19:34:43 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 19:34:53 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 19:34:54 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-06 19:34:57 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 19:34:57 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embedd

In [11]:
# Retrieval and generation: Let's set up a RAG chain (with whatever we've got)

from pydantic import Field
from langchain_ollama import OllamaLLM
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate

class CustomGraphitiRetriever(BaseRetriever):
    graphiti: Graphiti = Field(...)  # required field
    k: int = 5  # optionally limit results

    class Config:
        arbitrary_types_allowed = True  # required for non-Pydantic types

    def _get_relevant_documents(self, query, *, run_manager):
        print(f'Getting relevant documents... query={query}')
        raise NotImplementedError("This retriever only supports async")

    async def _aget_relevant_documents(self, query, *, run_manager):
        results = await graphiti.search(query)
        for res in results:
            print(res.fact)
        return [Document(page_content=node.fact) for node in results]

prompt = PromptTemplate.from_template('''
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. A longer answer isn't necessarily better. Use three sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
''')

llm = OllamaLLM(model="llama3.2:1b")

rag_chain = (
        {
            "context": CustomGraphitiRetriever(graphiti=graphiti),
            "question": RunnablePassthrough(),
        }
        | prompt
        | llm
        | StrOutputParser()
)

query = "What is the relationship between a prefix sum and an integral image?"
async def call_chain():
    return await rag_chain.ainvoke(query)

rag_response = await call_chain()
print(rag_response)

2025-08-06 20:15:41 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-06 20:15:41 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


Heapsort $n \log n$
Heapsort $n \log n$
Generally, priority queues share the same applications as heaps
Self-balancing binary search tree $n \log n$
Equivalence of priority queues and sorting algorithms
n is a number
Insertion sort $n$
Insertion sort $n$
Basic - enqueue: add an element to the queue with an associated priority.
is_empty: check whether the queue has no elements.
I don't know what the relationship between a prefix sum and an integral image is in this context.


In [ ]:
# whelp... 10 chunks in 45 minutes... that's prohibitive... Looks like I'm not going to be using Graphiti for QuizBot
#   - Still, it was interesting to explore and seems promising:
#       - Building knowledge graphs from unstructured data using LLMs has potential.
#       - Hooking up agents to KGs like Graphiti via MCP to give them a sort of "long-term memory" is an interesting research direction.
#   - I can still use a simple knowledge graph, like Neo4j by itself, to store my raw documents for multi-index retrieval
#       - The structure of an Obsidian vault naturally lends itself to being stored in a knowledge graph
#           - (In fact, that's the whole selling point of Obsidian)
#       - So I'm thinking a good process might be:
#           1. I'll do similarity searches against a vector database (FAISS)
#           2. I'll include a multi-index id in the chunk documents I retrieve
#           3. I can then use that id to look up the parent document in the knowledge graph (Neo4J)
#           4. From there, I can do knowledge graph shenanigans (center node search, etc.) to retrieve related documents